Select events from a catalogue based on the fault they're on

In [1]:
#import relevant modules
from rsqsim_api.catalogue.catalogue import RsqSimCatalogue
from rsqsim_api.fault.multifault import RsqSimMultiFault
import fnmatch
import os

In [27]:
# Tell python where field paths etc are relative to
script_dir = os.path.abspath('')
fault_dir = "../../../data/shaw2021/rundir5091"
catalogue_dir = fault_dir
outdir=os.path.join(catalogue_dir,"by_fault")

Read in faults and earthquake catalogue.  The files are quite large and can be downloaded from https://doi.org/10.5281/zenodo.5534462
The models have a spin up period so it's best to remove the first ~2e12s (60kyr) - see below.

In [5]:
fault_model = RsqSimMultiFault.read_fault_file_bruce(os.path.join(script_dir, fault_dir, "zfault_Deepen.in"),
                                                     os.path.join(script_dir, fault_dir, "znames_Deepen.in"),
                                                     transform_from_utm=True)
whole_catalogue = RsqSimCatalogue.from_catalogue_file_and_lists(os.path.join(catalogue_dir, "eqs..out"),
                                                                list_file_directory=catalogue_dir, list_file_prefix="catalog")

Remove first 2e12s

In [7]:
no_spin_up_cat=whole_catalogue.filter_whole_catalogue(min_t0=2.0e12)

Select faults of interest (the possible fault names can be listed using fault_model.names)

In [23]:
#e.g. single fault
faultName="alpinef2k"
fault_selection1=RsqSimMultiFault([fault_model.name_dic[faultName]])
#e.g. all faults with a particular string in the name
baseFault="hope"
faults2select = [name for name in fault_model.names if fnmatch.fnmatch(name, baseFault+"*")]
fault_selection2=RsqSimMultiFault([fault_model.name_dic[name] for name in faults2select])
#e.g. a list of faults
faultNames=["fidget","jorkekneed","clarencene"]
fault_selection3=RsqSimMultiFault([fault_model.name_dic[name] for name in faultNames])

#check these have worked correctly
#fault_selection1.names

Extract events on these faults.

In [36]:
event_selection1=no_spin_up_cat.filter_by_fault(fault_selection1, minimum_patches_per_fault=1)
event_selection2=no_spin_up_cat.filter_by_fault(fault_selection2, minimum_patches_per_fault=1)
event_selection3=no_spin_up_cat.filter_by_fault(fault_selection3, minimum_patches_per_fault=1)

Write out to new catalogues.

In [37]:
#first make sure output directory exists
if not os.path.exists(outdir):
    os.mkdir(outdir)

#write out
event_selection1.write_csv_and_arrays(prefix=faultName,directory=outdir)
event_selection2.write_csv_and_arrays(prefix=baseFault+"_all",directory=outdir)
#might want to change naming convention here especially for a long list
#currently just concatenates all list elements to get prefix
event_selection3.write_csv_and_arrays(prefix=''.join(faultNames[:]),directory=outdir)

writing
writing
writing


Example of how to read one of these catalogues back in:

In [38]:
alp_cat=RsqSimCatalogue.from_csv_and_arrays(prefix=os.path.join(outdir,faultName))